## 2018년 05월 18일 15시 09분 36초에 추가 ##
- Training FCN_RGB on NYUDv2

In [1]:
%matplotlib inline
import os
import collections
import torch
import torchvision
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt

from NYUDv2Loader import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
data_path = '/home/dongwonshin/Desktop/Datasets/NYUDv2/'
arg_string = '--arch FCN_RGB --batch_size 3 --n_epoch 50'

# Argument setting

In [2]:
import sys, os
import torch
import visdom
import argparse
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

from torch.autograd import Variable
from torch.utils import data
from tqdm import tqdm

from ptsemseg.models import get_model
from ptsemseg.loader import get_loader, get_data_path
from ptsemseg.metrics import runningScore
from ptsemseg.loss import *
from ptsemseg.augmentations import *


parser = argparse.ArgumentParser(description='Hyperparams')
parser.add_argument('--arch', nargs='?', type=str, default='fcn8s', help='Architecture to use [\'fcn8s, unet, segnet etc\']')
parser.add_argument('--img_rows', nargs='?', type=int, default=256, help='Height of the input image')
parser.add_argument('--img_cols', nargs='?', type=int, default=256, help='Width of the input image')

parser.add_argument('--img_norm', dest='img_norm', action='store_true', help='Enable input image scales normalization [0, 1] | True by default')
parser.add_argument('--no-img_norm', dest='img_norm', action='store_false', help='Disable input image scales normalization [0, 1] | True by default')
parser.set_defaults(img_norm=True)

parser.add_argument('--n_epoch', nargs='?', type=int, default=10, help='# of the epochs')
parser.add_argument('--batch_size', nargs='?', type=int, default=1, help='Batch Size')
parser.add_argument('--l_rate', nargs='?', type=float, default=1e-5, help='Learning Rate')
parser.add_argument('--feature_scale', nargs='?', type=int, default=1, help='Divider for # of features to use')
parser.add_argument('--resume', nargs='?', type=str, default=None, help='Path to previous saved model to restart from')

parser.add_argument('--visdom', dest='visdom', action='store_true', help='Enable visualization(s) on visdom | False by default')
parser.add_argument('--no-visdom', dest='visdom', action='store_false', help='Disable visualization(s) on visdom | False by default')
parser.set_defaults(visdom=False)

# Model init

In [3]:
args = parser.parse_args(arg_string.split(' '))

# Setup Dataloader
t_loader = NYUDv2Loader(data_path, is_transform=True)
v_loader = NYUDv2Loader(data_path, is_transform=True, split='val')

n_classes = t_loader.n_classes
trainloader = data.DataLoader(t_loader, batch_size=args.batch_size, num_workers=16, shuffle=True)
valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=16)

# Setup Metrics
running_metrics = runningScore(n_classes)

# Setup visdom for visualization
if args.visdom:
    vis = visdom.Visdom()

    loss_window = vis.line(X=torch.zeros((1,)).cpu(),
                       Y=torch.zeros((1)).cpu(),
                       opts=dict(xlabel='minibatches',
                                 ylabel='Loss',
                                 title='Training Loss',
                                 legend=['Loss']))

# Setup Model
model = get_model(args.arch, n_classes)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.cuda()

# Check if model has custom optimizer / loss
if hasattr(model.module, 'optimizer'):
    optimizer = model.module.optimizer
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=args.l_rate, momentum=0.99, weight_decay=5e-4)

if hasattr(model.module, 'loss'):
    print('Using custom loss')
    loss_fn = model.module.loss
else:
    loss_fn = cross_entropy2d

if args.resume is not None:                                         
    if os.path.isfile(args.resume):
        print("Loading model and optimizer from checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        print("Loaded checkpoint '{}' (epoch {})"                    
              .format(args.resume, checkpoint['epoch']))
    else:
        print("No checkpoint found at '{}'".format(args.resume)) 


# Training

In [4]:
best_iou = -100.0 
for epoch in range(args.n_epoch):
    model.train()
    for i, (color_imgs, depth_imgs, label_imgs) in enumerate(trainloader):
        color_imgs = Variable(color_imgs.cuda())
        depth_imgs = Variable(depth_imgs.cuda())
        label_imgs = Variable(label_imgs.cuda())

        optimizer.zero_grad()
        outputs = model(color_imgs)

        loss = loss_fn(input=outputs, target=label_imgs)

        loss.backward()
        optimizer.step()

        if args.visdom:
            vis.line(
                X=torch.ones((1, 1)).cpu() * i,
                Y=torch.Tensor([loss.data[0]]).unsqueeze(0).cpu(),
                win=loss_window,
                update='append')

        if (i+1) % 100 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, args.n_epoch, loss.data[0]))

    model.eval()
    for i_val, (color_images_val, depth_images_val, label_images_val) in tqdm(enumerate(valloader)):
        color_images_val = Variable(color_images_val.cuda(), volatile=True)
        depth_images_val = Variable(depth_images_val.cuda(), volatile=True)
        label_images_val = Variable(label_images_val.cuda(), volatile=True)

        outputs = model(color_images_val)
        pred = outputs.data.max(1)[1].cpu().numpy()
        gt = label_images_val.data.cpu().numpy()
        running_metrics.update(gt, pred)

    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
    running_metrics.reset()

    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t']
        state = {'epoch': epoch+1,
                 'model_state': model.state_dict(),
                 'optimizer_state' : optimizer.state_dict(),}
        torch.save(state, "../model_weights/{}_{}_best_model.pkl".format(args.arch, 'NYUDv2'))

/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/50] Loss: 2.2732
Epoch [1/50] Loss: 1.9622


218it [00:25,  8.67it/s]


Overall Acc: 	 0.342780395881
Mean Acc : 	 0.134631616316
FreqW Acc : 	 0.17698283708
Mean IoU : 	 0.0716676765573
Epoch [2/50] Loss: 2.2530
Epoch [2/50] Loss: 1.7566


218it [00:25,  8.56it/s]


...

218it [00:25,  8.47it/s]

Overall Acc: 	 0.572238791929
Mean Acc : 	 0.442995109541
FreqW Acc : 	 0.405022677128
Mean IoU : 	 0.321354635472


Epoch [50/50] Loss: 0.7283
Epoch [50/50] Loss: 0.7384


218it [00:25,  8.68it/s]


Overall Acc: 	 0.576334471163
Mean Acc : 	 0.468214265835
FreqW Acc : 	 0.410716455771
Mean IoU : 	 0.330127568796


# Training FCN_RGB on NYUDv2

# Second round

In [1]:
from utils import *

arg_str = '--gpu_idx 0 ' \
          '--arch FCN_RGB ' \
          '--dataset NYUDv2 ' \
          '--batch_size 3 ' \
          '--n_epoch 50 ' \
          '--resume ../model_weights/FCN_RGB_NYUDv2_best_model.pkl ' \
          '--visdom'

trainer = trainer(arg_str)
trainer.model_init()
trainer.training()

Loading model and optimizer from checkpoint '../model_weights/FCN_RGB_NYUDv2_best_model.pkl'
Loaded checkpoint '../model_weights/FCN_RGB_NYUDv2_best_model.pkl' (epoch 50)


/home/dongwonshin/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1462: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.upsample instead.")


Epoch [1/50] Loss: 0.8969
Epoch [1/50] Loss: 1.0645


218it [00:24,  8.75it/s]


Overall Acc: 	 0.574953854684
Mean Acc : 	 0.455462676677
FreqW Acc : 	 0.406740557506
Mean IoU : 	 0.324854994087
Epoch [2/50] Loss: 0.9917
Epoch [2/50] Loss: 1.0931


...

Epoch [49/50] Loss: 0.6590
Epoch [49/50] Loss: 0.6924


218it [00:25,  8.65it/s]


Overall Acc: 	 0.583667684044
Mean Acc : 	 0.483018677581
FreqW Acc : 	 0.418539167729
Mean IoU : 	 0.349213968762
Epoch [50/50] Loss: 0.7339
Epoch [50/50] Loss: 0.7823


218it [00:24,  8.99it/s]


Overall Acc: 	 0.580659095072
Mean Acc : 	 0.486402725488
FreqW Acc : 	 0.417790518162
Mean IoU : 	 0.349844535847
